# Lab06 - Preprocessing I
#### Name: Matt McLaughlin 
#### Class: CSCI 349 - Intro to Data Mining 
#### Semester: 2020SP 
#### Instructor: Brian King

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

**1) [P] Use pandas to read in your CSV data file you downloaded above, which you should have placed in your
data directory. Call the data frame df_temps. Read in the entire dataset, however, peek at the dataset
first. You'll notice 16 rows of metadata. Ignore the first 16 rows (HINT: Use the skiprows= option!)**

In [2]:
df_temps = pd.read_csv(filepath_or_buffer="../data/faa_hourly-KIPT_20000101-20191231_raw.csv", skiprows=16)
#df_temps

**NOTE: BE SURE TO LOOK AT YOUR ACTUAL DATA BEFORE TRYING TO READ IN A RAW DATASET! JUST BECAUSE
A DATASET HAS A .CSV EXTENSION DOES NOT MEAN THAT YOU CAN RELY ON EVERY ROW BEING A PROPERLY
FORMATTED ROW! For instance, notice that the header row is scattered throughout your data! Notice that
you have some extra columns at the end that are consistently empty! The inexperienced are tempted to
manually edit the file to make it easy to read. NO. WRONG! BAD DATA SCIENTIST! Write your Python cleaning
code to always work with raw, uncleaned data. Why? In practice, your data file may be huge. You may need to
repeatedly grab fresh data, that will only have the same issues. Do you really want to repeat your manual
editing silliness every time you have a fresh file? No! It may take a bit more work up front, but ALWAYS strive
to write code to preprocess every aspect of your data file! It will always save you work later!**

**2) Report the general structure of the data frame using df_temps.info(). You should notice that almost
every variable was read in as a plan object data type. You have a lot of work to do!
Your result should look as follows:**

In [3]:
df_temps.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 181914 entries, 0 to 181913
Data columns (total 14 columns):
Date/Time (GMT)                  181914 non-null object
Number of Observations (n/a)     181914 non-null object
Average Temp (F)                 180915 non-null object
Max Temp (F)                     180915 non-null object
Min Temp (F)                     180915 non-null object
Average Dewpoint Temp (F)        180809 non-null object
1 Hour Precip (in)               37596 non-null object
Max Wind Gust (mph)              32206 non-null object
Average Relative Humidity (%)    177416 non-null object
Average Wind Speed (mph)         181372 non-null object
Average Station Pressure (mb)    181636 non-null object
Average Wind Direction (deg)     149252 non-null object
Max Wind Speed (mph)             181372 non-null object
Unnamed: 13                      0 non-null float64
dtypes: float64(1), object(13)
memory usage: 19.4+ MB


**This is a pretty good dataset with lots of real problems! It gives you a chance to understand how important it is to
select the smallest, yet most accurate data type for every variable. This is particularly true with respect to your
memory footprint. With enormous data involving millions of records, you often need to perform various paging
exercises to load in chunks of data into memory, substantially slowing down the machine learning methods. In
other words, the more data you can fit in memory, the better!**

**3) [P] Read about the memory_usage() method of pandas data frames. Then, report the total memory in
bytes for each variable of df_temps. Set the parameter deep=True, to get the most accurate assessment
of your total memory usage. (NOTE – this could take a bit of time to return an answer.)**

In [4]:
#http://lira.no-ip.org:8080/doc/python-pandas-doc/html/generated/pandas.DataFrame.memory_usage.html#pandas.DataFrame.memory_usage
mem_use = df_temps.memory_usage(deep=True)
mem_use

Index                                 128
Date/Time (GMT)                  13790816
Number of Observations (n/a)     11477885
Average Temp (F)                 11342670
Max Temp (F)                     11308038
Min Temp (F)                     11308017
Average Dewpoint Temp (F)        11411051
1 Hour Precip (in)                7032800
Max Wind Gust (mph)               6907372
Average Relative Humidity (%)    11354487
Average Wind Speed (mph)         11291210
Average Station Pressure (mb)    11647992
Average Wind Direction (deg)     10569351
Max Wind Speed (mph)             11259415
Unnamed: 13                       1455312
dtype: int64

**4) [P] Report the total memory required for the data frame in MB. (Just sum the previous answer.) You should
get an answer showing over a hundred megabytes! Also, store the total as a variable called
original_memory. We're going to compare memory after we're done.**

In [5]:
original_memory = sum(mem_use) / 1048576
print("Original memory is " + "{:.3f}".format(original_memory) + " MB (rounded)")

Original memory is 135.571 MB (rounded)


**5) [P] Remember those extra column header lines that appeared throughout the entire CSV file? You need to get
rid of those. Write the code to eliminated those from df_temps. (This is tricky. Think about it... you are
selecting the data that does NOT have columns[0] as the first value in the observation!)
HINT – At this point, you should have 173252 observations.**

In [6]:
#I don't know why, but it took me an incredibly long time to get this. Just wasn't clicking.
fc = df_temps.columns[0]
df_temps = df_temps[df_temps[fc] != fc]
len(df_temps)

173252

**6) [M-P] Examine the last entry of your index (i.e. df_temps.index[-1]). Then, show the number of
observations in df_temps. These are unequal. Why?**

In [7]:
df_temps.index[-1]

181913

In [8]:
df_temps.shape[0]

173252

#### Because while we have been removing rows from the dataframe, we haven't been relabeling/renumbering the indexes.

**7) When you permanently delete observations, it's usually a good idea to reset your index, especially if the index
is nothing more than a unique number to each observations. Reindex your data, and show that the new index
is indeed reset. (There are many ways to do this. I suggest using reset_index(). There is no need to
retain the original index, so drop=True is fine.)**

In [9]:
df_temps = df_temps.reset_index(drop=True)

**8) You have a rather annoying extra column that was read in in the last column position. (Look closely at the
output of `describe()` above!) You should always confirm that it's garbage before deleting it. Write the
single line of code that reports the count of valid values in the last column (HINT: count())**

In [10]:
df_temps.count()["Unnamed: 13"]

0

**9) Drop that last column from df_temps.
I cannot emphasize this enough – you will get the most out of your data when you take the time to set up the
most accurate type for each variable. Currently, the type of every variable is object. However, notice that in
your raw data file, EVERY variable is a number except the first variable, which is a date. Dates are COMMON in
data, and it is important that you represent dates as actual date types! We'll deal with that shortly. Let's
continue cleaning this up.**

In [11]:
df_temps = df_temps.iloc[:,0:-2]

**10) [P] Convert all numeric data to actual numeric data types. You'll need to look up how to do this. (HINT:
pd.to_numeric() is your friend.) Leave the NaN fields alone! The fact that they are missing is
IMPORTANT! And, leave the date/time variable in the first column alone.
You should output the shape of your data, and show info() to show every variable is a floating point
number except the date/time field in the first column, which should still be of type object.**

In [12]:
#Convert numerica data to numeric data types
#df_temps = df_temps.iloc[1:,:].apply(pd.to_numeric)
#Not sure if theres a difference between the above and 
#df_temps = df_temps.apply(pd.to_numeric)

#Problem with all above is they don't pass parameters to pd.to_numeric(), causes trouble in #12
for i in range(1, len(df_temps.columns)):
    df_temps.iloc[:,i] = pd.to_numeric(arg=df_temps.iloc[:,i])

In [13]:
#Show shape()
df_temps.shape

(173252, 12)

In [14]:
#Show info() to show type of variables
df_temps.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 173252 entries, 0 to 173251
Data columns (total 12 columns):
Date/Time (GMT)                  173252 non-null object
Number of Observations (n/a)     173252 non-null int64
Average Temp (F)                 172253 non-null float64
Max Temp (F)                     172253 non-null float64
Min Temp (F)                     172253 non-null float64
Average Dewpoint Temp (F)        172147 non-null float64
1 Hour Precip (in)               28934 non-null float64
Max Wind Gust (mph)              23544 non-null float64
Average Relative Humidity (%)    168754 non-null float64
Average Wind Speed (mph)         172710 non-null float64
Average Station Pressure (mb)    172974 non-null float64
Average Wind Direction (deg)     140590 non-null float64
dtypes: float64(10), int64(1), object(1)
memory usage: 15.9+ MB


**11) [P] How much did our memory footprint improve? (Show the total memory usage using deep=True). Report
the total memory usage in MB, and report the percentage improvement.**

In [15]:
mem_use2 = df_temps.memory_usage(deep=True)
mem_use2

Index                                 128
Date/Time (GMT)                  13167152
Number of Observations (n/a)      1386016
Average Temp (F)                  1386016
Max Temp (F)                      1386016
Min Temp (F)                      1386016
Average Dewpoint Temp (F)         1386016
1 Hour Precip (in)                1386016
Max Wind Gust (mph)               1386016
Average Relative Humidity (%)     1386016
Average Wind Speed (mph)          1386016
Average Station Pressure (mb)     1386016
Average Wind Direction (deg)      1386016
dtype: int64

In [16]:
new_memory = sum(mem_use2) / 1048576
print("New memory is " + "{:.3f}".format(new_memory) + " MB (rounded)")

print("Improvment over original: " + "{:.4f}".format((original_memory-new_memory)/original_memory) + "% (rounded)")

New memory is 27.097 MB (rounded)
Improvment over original: 0.8001% (rounded)


**12) [P] Did you notice that to_numeric() has a parameter called downcast? Go back and read about this
parameter. By default, most of the time your integer types will be converted to a 64-bit integer, and floating point types will use double precision numbers. You can do far better. Downcast your types accordingly. Report your latest memory usage in MB.**

In [17]:
#Downcast types to less memory usage
df_temps.iloc[:,1] = pd.to_numeric(arg=df_temps.iloc[:,1], downcast='integer')
for i in range(2, len(df_temps.columns)):
    df_temps.iloc[:,i] = pd.to_numeric(arg=df_temps.iloc[:,i], downcast='float')

In [18]:
#Report latest memory usage
new_m2 = sum(df_temps.memory_usage(deep=True)) / 1048576
print("New memory is " + "{:.3f}".format(new_m2) + " MB (rounded)")
print("Improvment over original: " + "{:.4f}".format((original_memory-new_m2)/original_memory) + "% (rounded)")

New memory is 19.332 MB (rounded)
Improvment over original: 0.8574% (rounded)


**13) At this point, with the exception of the date column, you should have good data to start working with. Verify it
by outputting the results of describe(). Every variable should have its basic stats reported!**

In [19]:
df_temps.describe()

,Number of Observations (n/a),Average Temp (F),Max Temp (F),Min Temp (F),Average Dewpoint Temp (F),1 Hour Precip (in),Max Wind Gust (mph),Average Relative Humidity (%),Average Wind Speed (mph),Average Station Pressure (mb),Average Wind Direction (deg)
count,173252.000000,172253.000000,172253.000000,172253.000000,172147.000000,28934.000000,23544.000000,168754.000000,172710.000000,172974.000000,140590.000000
mean,1.339915,51.286217,51.399208,51.179626,40.253628,0.030566,22.295341,68.819511,5.904806,1016.741272,176.806366
std,0.854852,18.888988,18.907930,18.881851,19.053423,0.079247,7.581788,19.694031,5.192565,7.640043,118.790619
min,0.000000,-11.900000,-11.900000,-11.900000,-20.900000,0.000000,0.000000,0.000000,0.000000,508.600006,0.000000
25%,1.000000,36.000000,36.000000,36.000000,25.830000,0.000000,19.600000,54.000000,0.000000,1012.200012,70.000000
50%,1.000000,52.000000,52.000000,51.799999,41.000000,0.000000,21.900000,71.000000,5.370000,1016.900024,220.000000
75%,1.000000,66.900002,66.900002,66.900002,57.000000,0.030000,26.500000,86.000000,9.200000,1021.700012,280.000000
max,10.000000,102.000000,102.000000,102.000000,79.000000,2.350000,88.599998,100.000000,76.000000,1044.400024,360.000000


**Data Transformation with Dates (a lot of text, see lab)**

**14) [M] There are four primary classes in pandas for working with dates and times? Consider the Scalar Class for
each, and state what concept each is representing.**

#### 1. Timestamp    Concept: date times- a specific date and time with timezone support
#### 2. Timedelta      Concept: time deltas- an absolute time duration
#### 3. Period            Concept: time spans- a span of time defined by a point in time and its associate frequency
####  4. DateOffset    Concept: date offsets- a relative time duration that respect calendar arithmetic

**15) [M] For each above, state the primary creation method used to create each type of data**

#### Timestamp: to_datetime or date_range
#### Timedelta: to_timedelta or timedelta_range
#### Time spans: Period or period_range
#### DateOffset: Dateoffset

**16) [P] Create a Timestamp object from the string "07/04/19", which is a date representing July 4, 2019.
Store the object as d1 and show it.**

In [20]:
d1 = pd.to_datetime("07/04/19")
d1

Timestamp('2019-07-04 00:00:00')

**17) [P] Using d1 and string formatting codes, print the string from d1:
"Today's date is Thursday, July 4, 2019".**

In [21]:
#I don't know if this is what you wanted...
print("Today's date is " + d1.day_name() + ", " + d1.month_name() + " " + str(d1.day) + ", " + str(d1.year))

Today's date is Thursday, July 4, 2019


**18) [P] Create another Timestamp object representing Sept 7, 2019 at 3pm, called d2. Report it**

In [22]:
d2 = pd.to_datetime("09/07/19 15:00:00")
d2

Timestamp('2019-09-07 15:00:00')

**19) [P] Subtract d2 – d1, and report the difference as the number of days and seconds between these two. Also
report the difference as total seconds. (NOTE: The difference should be 65 days, 54000 seconds. Or 5670000
total seconds.)**

In [23]:
d19 = d2 - d1 #Implicitly creates a timedelta object
print("Difference is " + str(d19.days) + " days and " + str(d19.seconds) + " seconds.")
print("Total seconds is " + str(d19.total_seconds()))

Difference is 65 days and 54000 seconds.
Total seconds is 5670000.0


**20) [P] Create a new Timestamp object from the string "2019-07-01 08:30pm", but, localize the time
stamp to represent the time in the US Eastern Time Zone. Store the result as d3 and output it.**

In [24]:
d3 = pd.to_datetime("2019-07-01 08:30pm").tz_localize("US/Eastern")
d3

Timestamp('2019-07-01 20:30:00-0400', tz='US/Eastern')

**21) [P] Show time represented by d3, but converted to the US / Pacific Time Zone. The time reported should be
three hours earlier than EST shown in the previous question.**

In [25]:
d3.tz_convert("US/Pacific")

Timestamp('2019-07-01 17:30:00-0700', tz='US/Pacific')

**22) [P] Create a Timestamp object representing right now, stored as ts_now. Show the result.**

In [26]:
ts_now = pd.Timestamp.now()
ts_now

Timestamp('2020-02-04 21:28:52.720375')

**23) [P] Create a Timedelta object representing 1 hour, stored as td_hour. Show the result.**

In [27]:
td_hour = pd.Timedelta(hours=1)
td_hour

Timedelta('0 days 01:00:00')

**24) [P] Demonstrate how you can do basic mathematical operations by adding 6 hours to ts_now using
td_hour and basic math operations. (i.e. No loops or further calculations necessary!)**

In [28]:
ts_now + 6*td_hour

Timestamp('2020-02-05 03:28:52.720375')

**25) [P] Create a DatetimeIndex object that represents every hour during the month of January, 2019. The first
index should be midnight, January 1, 2020, and the last index should be January 31, 2020 at 11pm. Store the
object as dr. (HINT – use the pd.date_range() method!)**

In [29]:
dr = pd.date_range(start=pd.to_datetime("01/01/2020 00:00:00"), end=pd.to_datetime("01/31/2020 23:00:00"), freq="H")

**OK, so that was a little practice with understanding how to work a bit with dates and times. They are objects, with
lots of methods to help you access those timestamps in different ways.
Back to our weather data. Usually, the index to a dataframe represents the data you will use most often to access
and select your data. In the case of a time series dataset, the index is usually the time. In other words, every
observation should be indexed by a Timestamp object! You'll make that happen next...**

**26) [P] The first variable in our data is currently an object. But, notice the name and its units? It's a date/time in
the GMT time zone! Convert the first column of data into an actual time stamp.
NOTE: You can NOT simply generate this column using your own date range object! You must generate it
directly from the actual time/date stamp in the data! Why? This is very important. Do NOT ever be fooled
into thinking any real-world dataset you are dealing with is 100% complete. There are missing observations
in these data, and your data will be massively flawed if you neglect this! If you simply try to use a date range
between 1/1 – 12/31, with every hour, you are making an incorrect assumption that every observation is
present.
(HINT: Go back to your reference table. You are creating an array of timestamps. Which function? Either
to_datetime or date_range. We already told you that date_range is wrong!)**

In [30]:
#use pd.to_datetime() on first column of df_temps
df_temps.iloc[:,0] = pd.to_datetime(df_temps.iloc[:,0])#.convert("US/GMT")

**27) [P] Confirm that your first column data type is now a timestamp by showing the output of
df_temps.info(). (It should show that it is datetime64, to be exact) . Then, show the values of the
first column of the first AND last row only. Your result should look like:
0 2000-01-01 00:00:00
173251 2019-12-31 23:00:00**

In [31]:
df_temps.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 173252 entries, 0 to 173251
Data columns (total 12 columns):
Date/Time (GMT)                  173252 non-null datetime64[ns]
Number of Observations (n/a)     173252 non-null int8
Average Temp (F)                 172253 non-null float32
Max Temp (F)                     172253 non-null float32
Min Temp (F)                     172253 non-null float32
Average Dewpoint Temp (F)        172147 non-null float32
1 Hour Precip (in)               28934 non-null float32
Max Wind Gust (mph)              23544 non-null float32
Average Relative Humidity (%)    168754 non-null float32
Average Wind Speed (mph)         172710 non-null float32
Average Station Pressure (mb)    172974 non-null float32
Average Wind Direction (deg)     140590 non-null float32
dtypes: datetime64[ns](1), float32(10), int8(1)
memory usage: 8.1 MB


In [32]:
df_temps.iloc[[0,-1],0]

0        2000-01-01 00:00:00
173251   2019-12-31 23:00:00
Name: Date/Time (GMT), dtype: datetime64[ns]

**28) Finally, let's move that first column to be the new index for your dataframe. Use the set_index method of
of df_temps to be the first column of data, then use the drop method to eliminate the first column. It is now your index, and thus there is no need to keep this information twice.**

In [33]:
df_temps = df_temps.set_index(df_temps.columns[0]) #drop=True by default

**29) [P] Give one final report on the total memory usage, and also show the % memory reduction made compared
to when you first loaded the data.
Again, please take this seriously. This is a substantial amount of memory saved! Why? Because you took the
time to properly process every column to have it represent its most accurate type, using the smallest type
necessary. HUGE savings!**

In [34]:
new_m29 = sum(df_temps.memory_usage(deep=True)) / 1048576
print("New memory is " + "{:.3f}".format(new_m29) + " MB (rounded)")
print("Improvment over original: " + "{:.4f}".format((original_memory-new_m29)/original_memory) + "% (rounded)")

New memory is 8.096 MB (rounded)
Improvment over original: 0.9403% (rounded)


**30) [P] This dataset has missing observations. But, how many? First, calculate how many observations SHOULD be
there. Use the difference between the first and last index value to compute this.**

In [35]:
d30 = df_temps.index[-1] - df_temps.index[0]
d31 = d30.total_seconds() / 3600
print("There should be " + str(d31) + " observations total")
print("There are " + str(d31 - 173251) + " missing") # Number is length of df_temps

There should be 175319.0 observations total
There are 2068.0 missing


**31) [P] There are quite a lot! It's time to investigate. Create a data frame called df_missing that has an index
of the time stamp of every missing date, with a simple variable called "missing" that has a value of 1 for every
entry. (i.e. it should only contain the missing dates.) Report the number of rows in df_missing. It should
match the number you computed previously.**

In [36]:
missing_dates = pd.date_range(start=df_temps.index[0], end=df_temps.index[-1], freq="H").difference(df_temps.index)

one_series = pd.Series(np.ones(len(missing_dates)), index=missing_dates, name="missing")
d = {'missing':one_series}
df_missing = pd.DataFrame(data=d)
print("Number of rows in df_missing: " + str(df_missing.shape[0])) #This is how many missing dates there were

Number of rows in df_missing: 2068


**32) [P] Let's get a sense of which years seem to be missing the most data. How? Well, the easiest approach is
probably to use the resample() method of data frames. Check out this section:
https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#resampling . This method works
phenomenally well for grouping and aggregating your data when you have a datetime index type!
We're going to resample our data by year, and perform a count aggregation all in one line:
Enter the following:
df_missing_by_year = df_missing.resample('Y').count()
There are many, many ways you can resample your data. You need to jump over to the options for dateoffset
objects. The letter codes are specified there:
https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#dateoffset-objects
Show the result of df_missing_by_year**

In [37]:
df_missing_by_year = df_missing.resample('Y').count()
df_missing_by_year

,missing
2000-12-31,792
2001-12-31,54
2002-12-31,30
2003-12-31,39
2004-12-31,72
2005-12-31,119
2006-12-31,32
2007-12-31,64
2008-12-31,193
2009-12-31,82


**33) [P] You can see that pretty much every year has missing data. Not uncommon. However, one year in particular
is really bad. Which one? Write the code to eliminate that entire year from df_temps.**

In [38]:
#This was surprisingly easy
bad_year = df_missing_by_year.idxmax()[0].year
df_temps = df_temps[df_temps.index.year != bad_year]
df_temps.head() #Starts in 2001

,Number of Observations (n/a),Average Temp (F),Max Temp (F),Min Temp (F),Average Dewpoint Temp (F),1 Hour Precip (in),Max Wind Gust (mph),Average Relative Humidity (%),Average Wind Speed (mph),Average Station Pressure (mb),Average Wind Direction (deg)
Date/Time (GMT),,,,,,,,,,,
2001-01-01 00:00:00,1,21.0,21.0,21.0,6.1,NaN,NaN,51.0,12.7,1019.000000,270.0
2001-01-01 01:00:00,1,19.9,19.9,19.9,7.0,NaN,NaN,56.0,10.4,1019.000000,270.0
2001-01-01 02:00:00,1,19.0,19.0,19.0,7.0,NaN,NaN,58.0,16.1,1019.000000,270.0
2001-01-01 03:00:00,1,19.0,19.0,19.0,7.0,NaN,NaN,58.0,15.0,1018.599976,260.0
2001-01-01 04:00:00,1,19.0,19.0,19.0,6.1,NaN,NaN,56.0,16.1,1018.599976,280.0
